In [4]:
%load_ext autoreload
%autoreload 2

from Shared.shared import *
from Shared.specific_CNB_sim import *

sim_name = f"Dopri5_1k"
sim_folder = f"sim_output/{sim_name}"
fig_folder = f"figures_local/{sim_name}"
Cl_folder = f"Shared/Cls"
nu_m_range = jnp.load(f"{sim_folder}/neutrino_massrange_eV.npy")
nu_m_picks = jnp.array([0.01, 0.05, 0.1, 0.2, 0.3])*Params.eV
simdata = SimData(sim_folder)

z_int_steps = jnp.load(f"{sim_folder}/z_int_steps.npy")
s_int_steps = jnp.load(f"{sim_folder}/s_int_steps.npy")

#! Broken halos: either snapshot info missing or anomalous number densities
exclude_nums = jnp.array([
    20,
    # 21,
    24,
    25,
])
halo_nums = [x for x in range(1, 31) if x not in exclude_nums]


# Fermi-Dirac distribution assumption
# note: total densities are computed for length 50 mass range
pix_dens_FD = jnp.load(f"{sim_folder}/pixel_densities.npy")[tuple(halo_nums),...]
tot_dens_FD = jnp.load(f"{sim_folder}/total_densities.npy")[tuple(halo_nums),...]

# pix_dens_FD = jnp.delete(pix_dens_FD, exclude_nums, axis=0)
# tot_dens_FD = jnp.delete(tot_dens_FD, exclude_nums, axis=0)
print(pix_dens_FD.shape)
print(tot_dens_FD.shape)

# Incl. primordial fluctuations, for all halos with 1 seed
# note: all sky densities are computed for 5 specific masses
pix_dens_PF_halos = jnp.load(f"{sim_folder}/pixel_densities_incl_PFs.npy")[tuple(halo_nums),...]
tot_dens_PF_halos = jnp.load(f"{sim_folder}/total_densities_incl_PFs.npy")[tuple(halo_nums),...]

# pix_dens_PF_halos = jnp.delete(pix_dens_PF_halos, exclude_nums, axis=0)
# tot_dens_PF_halos = jnp.delete(tot_dens_PF_halos, exclude_nums, axis=0)
print(pix_dens_PF_halos.shape)
print(tot_dens_PF_halos.shape)

# Incl. primordial fluctuations, for all halos with 1 seed
# note: all sky densities are computed for 5 specific masses
pix_dens_PF_seeds = jnp.array(
    [jnp.load(f"{sim_folder}/pixel_densities_incl_PFs_seeds_halo{halo_j}.npy")
     for halo_j in halo_nums])
tot_dens_PF_seeds = jnp.array(
    [jnp.load(f"{sim_folder}/total_densities_incl_PFs_seeds_halo{halo_j}.npy")
     for halo_j in halo_nums])
print(pix_dens_PF_seeds.shape)
print(tot_dens_PF_seeds.shape)

# Deltas_halos = jnp.load(f"{sim_folder}/Deltas_halos.npy")
# print(Deltas_halos.shape)

# Deltas_seeds = jnp.load(f"{sim_folder}/Deltas_seeds.npy")
# print(Deltas_seeds.shape)


#! some change/bug in the sim_execute script, now FD also only has 5 masses instead of 50.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(27, 5, 768)
(27, 50)
(27, 5, 768)
(27, 5)
(27, 10, 5, 768)
(27, 10, 5)


## Sim output processing

In [13]:
# The decay simulation array which contains all the 0s and 1s for the neutrinos
# at each redshift step needs to be processed in the following way:
# 1. Reshape to all-sky type, i.e. smth like (zeds, Npix, nu_per_pix)
# 2. Compute percentage of decayed neutrinos in each pixel

# Load 
data_sim_folder = f"sim_output/no_gravity_decay"
Gamma_str = "2T"
decayed_neutrinos_z = jnp.load(
    f'{data_sim_folder}/decayed_neutrinos_z_{Gamma_str}.npy'
).reshape(-1, simdata.Npix, simdata.p_num)  # 0s and 1s
print(decayed_neutrinos_z.shape)

perc_parents_per_pix = jnp.sum(decayed_neutrinos_z, axis=-1)/simdata.p_num
perc_daughters_per_pix = 1-perc_parents_per_pix
print(perc_per_pix.shape, perc_per_pix[-1,::20]*Params.N0_cm3)

# Plotting all-sky map for parents:
# (Pixel value for no-decay sim) x (1 - decay perc. for that pixel)

# Plotting all-sky map for daughters:
# (Pixel value for no-decay sim) x (decay perc. for that pixel)

(105, 768, 1000)
(105, 768) [25.92697772 24.45514055 25.87036859 25.24766826 24.28531319 25.92697772
 25.98358684 26.1534142  25.64393211 26.32324157 24.62496792 26.26663245
 25.47410474 26.32324157 24.51174968 28.19134258 26.43645981 26.21002332
 26.43645981 25.07784089 26.09680508 25.87036859 26.09680508 24.28531319
 25.41749562 27.00255103 25.75715035 25.24766826 26.54967805 26.21002332
 23.49278549 25.3608865  27.34220576 24.90801353 27.56864224 27.11576927
 25.19105913 26.09680508 25.02123177]


## Power Spectra

### Parents

### Daughters